In [24]:
# Impor the IUCN Red List Api Key
from iucn_api_key import token

In [6]:
# Dependencies
import json
import requests 
import pandas as pd
from pprint import pprint

### Import Countries

In [35]:
# Define the API URL
url = f"https://apiv3.iucnredlist.org/api/v3/country/list?token={token}"
print (url)
# Make the request
response = requests.get(url)
print(response.status_code)
#print(response.json())

https://apiv3.iucnredlist.org/api/v3/country/list?token=9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee
200


In [23]:
response_json = response.json()
pprint(response_json)

{'count': 251,
 'results': [{'country': 'Uzbekistan', 'isocode': 'UZ'},
             {'country': 'Qatar', 'isocode': 'QA'},
             {'country': 'Saudi Arabia', 'isocode': 'SA'},
             {'country': 'Afghanistan', 'isocode': 'AF'},
             {'country': 'Lebanon', 'isocode': 'LB'},
             {'country': 'Bahrain', 'isocode': 'BH'},
             {'country': 'Cyprus', 'isocode': 'CY'},
             {'country': 'Syrian Arab Republic', 'isocode': 'SY'},
             {'country': 'Tajikistan', 'isocode': 'TJ'},
             {'country': 'Turkmenistan', 'isocode': 'TM'},
             {'country': 'United Arab Emirates', 'isocode': 'AE'},
             {'country': 'Iran, Islamic Republic of', 'isocode': 'IR'},
             {'country': 'Iraq', 'isocode': 'IQ'},
             {'country': 'Oman', 'isocode': 'OM'},
             {'country': 'Pakistan', 'isocode': 'PK'},
             {'country': 'Israel', 'isocode': 'IL'},
             {'country': 'Jordan', 'isocode': 'JO'},
             

### Countries and ISO Codes

In [43]:
# Initialize a list to store country data and iso_codes
# countries = []
iso_codes = []

# Define the API URL
url = f"https://apiv3.iucnredlist.org/api/v3/country/list?token={token}"
print (url)
# Make the request
response = requests.get(url)
response_json = response.json()

country_list = []

# Extract country names and ISO codes
for country_data in response_json.get('results', []):
    country_name = country_data.get('country')
    iso_code = country_data.get('isocode')
    country_list.append({
                'country_name': country_name,
                'iso_code': iso_code
            })
    #countries.append(country_name)
    iso_codes.append(iso_code)
df_countries = pd.DataFrame(country_list)
df_countries
#pprint(countries)
#pprint(iso_codes)
len(iso_codes)

https://apiv3.iucnredlist.org/api/v3/country/list?token=9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee


251

### Get countries and taxonid

In [ ]:
# List of ISO codes to process
# iso_code_test = ['AZ', 'QA']

# Initialize a list to store species data
data_list = []
for iso_code in iso_codes:
# Define the API URL
    url = f"https://apiv3.iucnredlist.org/api/v3/country/getspecies/{iso_code}?token={token}"

    # Make the request
    response = requests.get(url)
    response_json = response.json()
        
    # Get the country code from the response JSON
    country_code = response_json.get('country', '')
        
    # Extract species data
    for item in response_json.get('result', []):
        taxonid = item.get('taxonid')
        data_list.append({
            'iso_code': country_code,
            'species_id': taxonid
        })
    
    #countries.append(country_name)
    #iso_codes.append(iso_code)
df_countries_species = pd.DataFrame(data_list)
df_countries_species
#pprint(countries)
#pprint(iso_codes)

### Get lists of Species IDs and Common Names

In [21]:
# Define the API URL
url = f"https://apiv3.iucnredlist.org/api/v3/species/page/0?token={token}"

# Make the request
response = requests.get(url)
response_json = response.json()

# Initialize an empty list to store dictionaries
species_data = []

# Loop through the results and create dictionaries
for item in response_json.get('result', []):
    common_name = item.get('main_common_name')
    taxonid = item.get('taxonid')
    
    # Only add to data if common_name is not None
    if common_name is not None:
        species_data.append({
            'species_id': taxonid,
            'common_name': common_name
        })

# Convert the list of dictionaries to a DataFrame
df_species = pd.DataFrame(species_data)

# Reset index
df_species = df_species.reset_index(drop=True)

# Display the DataFrame
print(df_species)

      species_id                             common_name
0              9                Mekong Giant Salmon Carp
1             18                 Bolivian Chinchilla Rat
2             20  Monte Cristo Arboreal Alligator Lizard
3             43                        Thylacine Darner
4             73                              Hula Bream
...          ...                                     ...
6830       31435                           Selangan Batu
6831       31436                          Yellow Meranti
6832       31448                     Silver-leaved Ebony
6833       31618                                  Sabina
6834       31619                                 Yararey

[6835 rows x 2 columns]


### Loop through pages of species

add in kingdom phylum class

In [27]:
# Define the base URL and token
base_url = "https://apiv3.iucnredlist.org/api/v3/species/page/"

# Initialize an empty list to store dictionaries
species_data = []

# Start with the first page
page = 0

while True:
    # Define the URL for the current page
    url = f"{base_url}{page}?token={token}"
    print(url)
    
    # Make the request
    response = requests.get(url)
    response_json = response.json()
    
    # Get the results
    results = response_json.get('result', [])
    
    # Break the loop if no results are returned
    if not results:
        break
    
    # Loop through the results and create dictionaries
    for item in results:
        common_name = item.get('main_common_name')
        taxonid = item.get('taxonid')
        category = item.get('category')
        
        # Only add to data if common_name is not None
        if common_name is not None:
            species_data.append({
                'species_id': taxonid,
                'common_name': common_name,
                'category': category
            })
    
    # Move to the next page
    page += 1

# Convert the list of dictionaries to a DataFrame
df_species = pd.DataFrame(species_data)

# Reset index to start from 1
df_species = df_species.reset_index(drop=True)

# Display the DataFrame
df_species.head()


https://apiv3.iucnredlist.org/api/v3/species/page/0?token=9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee
https://apiv3.iucnredlist.org/api/v3/species/page/1?token=9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee
https://apiv3.iucnredlist.org/api/v3/species/page/2?token=9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee
https://apiv3.iucnredlist.org/api/v3/species/page/3?token=9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee
https://apiv3.iucnredlist.org/api/v3/species/page/4?token=9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee
https://apiv3.iucnredlist.org/api/v3/species/page/5?token=9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee
https://apiv3.iucnredlist.org/api/v3/species/page/6?token=9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee
https://apiv3.iucnredlist.org/api/v3/species/page/7?token=9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee
https://apiv3.iu

In [33]:
df_species.head()

,species_id,common_name,category
0,9,Mekong Giant Salmon Carp,CR
1,18,Bolivian Chinchilla Rat,CR
2,20,Monte Cristo Arboreal Alligator Lizard,EN
3,43,Thylacine Darner,EN
4,73,Hula Bream,EX


In [13]:
url = f"https://apiv3.iucnredlist.org/api/v3/threats/species/id/181008073?token={token}"
#https://apiv3.iucnredlist.org/api/v3/threats/species/id/181008073?token=9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee
print(url)
response = requests.get(url)
response_json = response.json()
pprint(response_json)
# # Get the species name
# species_name = response_json.get('name', '')  
# print(species_name)
        
# # Get the threat data from the response
# threat_data = response_json.get('result', [])

# # Add the species name to each threat entry
# for threat in threat_data:
#     threat['species_name'] = species_name
#     species_threat_data.append(threat)

https://apiv3.iucnredlist.org/api/v3/threats/species/id/181008073?token=9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee
{'id': '181008073',
 'result': [{'code': '1.1',
             'invasive': None,
             'scope': 'Minority (<50%)',
             'score': 'Unknown',
             'severity': 'Unknown',
             'timing': 'Ongoing',
             'title': 'Housing & urban areas'},
            {'code': '11.2',
             'invasive': None,
             'scope': 'Majority (50-90%)',
             'score': 'Medium Impact: 6',
             'severity': 'Slow, Significant Declines',
             'timing': 'Ongoing',
             'title': 'Droughts'},
            {'code': '11.5',
             'invasive': None,
             'scope': 'Whole (>90%)',
             'score': 'Unknown',
             'severity': 'Unknown',
             'timing': 'Ongoing',
             'title': 'Other impacts'},
            {'code': '1.2',
             'invasive': None,
             'scope': '